# M2177.003100 Deep Learning <br> Final Proejct: Text to Image Synthesis (Tensorflow)

Copyright (C) Data Science & AI Laboratory, Seoul National University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them. 

**For understanding of this work, please carefully look at given PPT file.**

**Note**: certain details are missing or ambiguous on purpose, in order to test your knowledge on the related materials. However, if you really feel that something essential is missing and cannot proceed to the next step, then contact the teaching staff with clear description of your problem.

### Submitting your work:
<font color=red>**DO NOT clear the training process **</font> so that TAs can grade both your code and results.  
**The TA will set a config file as 'eval_birds.yml' when evaluating the code using 'hidden test dataset'. Thus, please make sure that your code can generate proper data to measure inception score and R-precision of 'hidden test dataset'.**

## 1. Load datasets
The Birds dataset will be downloaded automatically if it is not located in the *data* directory. <br>

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os, nltk
from miscc.config import cfg, cfg_from_file
import pprint
import datetime
import dateutil.tz
import numpy as np
import scipy
from utils.data_utils import CUBDataset
from utils.loss import cosine_similarity
import pandas as pd
from scipy.io import loadmat
import re
import string
import random
import time

%matplotlib inline

/home/chszerg/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/chszerg/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/chszerg/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/chszerg

In [2]:
# Set a config file as 'train_birds.yml' in training, as 'eval_birds.yml' for evaluation
cfg_from_file('cfg/train_birds.yml') # eval_birds.yml

print('Using config:')
pprint.pprint(cfg)

os.environ['CUDA_VISIBLE_DEVICES'] = cfg.GPU_ID

now = datetime.datetime.now(dateutil.tz.tzlocal())
timestamp = now.strftime('%Y_%m_%d_%H_%M_%S')
output_dir = 'sample/%s_%s_%s' % (cfg.DATASET_NAME, cfg.CONFIG_NAME, timestamp)

Using config:
{'BATCH_SIZE': 64,
 'CHECKPOINT_DIR': './checkpoint',
 'CHECKPOINT_NAME': 'model.ckpt',
 'CNN': {'EMBEDDING_DIM': 0, 'H_DIM': 0},
 'CONFIG_NAME': 'text-to-image',
 'CUDA': False,
 'DATASET_NAME': 'birds',
 'DATA_DIR': 'data/birds',
 'EMBEDDING_TYPE': 'cnn-rnn',
 'GAN': {'B_ATTENTION': False,
         'B_CONDITION': False,
         'B_DCGAN': False,
         'CONDITION_DIM': 0,
         'DF_DIM': 0,
         'EMBEDDING_DIM': 0,
         'GF_DIM': 0,
         'R_NUM': 0,
         'Z_DIM': 512},
 'GPU_ID': '0',
 'IMAGE_SIZE': 256,
 'NUM_BATCH_FOR_TEST': 0,
 'RANDOM_SEED': 0,
 'RNN': {'EMBEDDING_DIM': 0,
         'H_DIM': 0,
         'TYPE': '',
         'VOCAB_SIZE': 0,
         'WORD_EMBEDDING_DIM': 0},
 'R_PRECISION_DIR': './evaluation',
 'R_PRECISION_FILE': 'r_precision.npz',
 'R_PRECISION_FILE_HIDDEN': 'r_precision_hidden.npz',
 'TEST': {'B_EXAMPLE': False,
          'GENERATED_HIDDEN_TEST_IMAGES': './evaluation/generated_images_hidden',
          'GENERATED_TEST_IMAGES'

/home/chszerg/final-project-deep-learning-19-tf/miscc/config.py:121: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  yaml_cfg = edict(yaml.load(f))


In [3]:
train_dataset = CUBDataset(cfg.DATA_DIR, split='train')
test_dataset = CUBDataset(cfg.DATA_DIR, split='test')

self.current_dir:
/home/chszerg/final-project-deep-learning-19-tf

self.data_dir:
/home/chszerg/final-project-deep-learning-19-tf/data/birds

self.image_dir:
/home/chszerg/final-project-deep-learning-19-tf/data/birds/CUB_200_2011.tgz

Dataset already exists
self.image_dir:
/home/chszerg/final-project-deep-learning-19-tf/data/birds/CUB_200_2011/images

Load from:  data/birds/captions.pickle
self.current_dir:
/home/chszerg/final-project-deep-learning-19-tf

self.data_dir:
/home/chszerg/final-project-deep-learning-19-tf/data/birds

self.image_dir:
/home/chszerg/final-project-deep-learning-19-tf/data/birds/CUB_200_2011.tgz

Dataset already exists
self.image_dir:
/home/chszerg/final-project-deep-learning-19-tf/data/birds/CUB_200_2011/images

Load from:  data/birds/captions.pickle


In [4]:
train_images = train_dataset.images
test_images = test_dataset.images
train_captions = np.asarray(train_dataset.captions_ids)
test_captions = np.asarray(test_dataset.captions_ids)
print(train_images.shape)
print(test_images.shape)
print(train_captions.shape)
print(test_captions.shape)

(8855, 256, 256, 3)
(2933, 256, 256, 3)
(88550, 20)
(29330, 20)


In [5]:
from skimage.transform import resize
train_images_64 = []
for train_image in train_images:
    train_images_64.append(resize(train_image, (64, 64, 3)))
train_images_64 = np.asarray(train_images_64)
print(train_images_64.shape)
assert train_images_64.shape[0] == train_images.shape[0]
test_images_64 = []
for test_image in test_images:
    test_images_64.append(resize(test_image, (64, 64, 3)))
test_images_64 = np.asarray(test_images_64)
print(test_images_64.shape)
assert test_images_64.shape[0] == test_images.shape[0]

(8855, 64, 64, 3)
(2933, 64, 64, 3)


In [6]:
n_captions_train = len(train_captions)
n_captions_per_image = 10
n_images_train = len(train_images)
print(train_images.shape)

(8855, 256, 256, 3)


In [72]:
import scipy.misc
import threading
import scipy.ndimage as ndi
from skimage import transform
from skimage import exposure
import skimage
from nltk.tokenize import RegexpTokenizer
import scipy.ndimage as ndimage

def sent2ID(sample_sentence):
    caption = []
    cap = sample_sentence
    if len(cap) == 0:
        exit()
    cap = cap.replace("\ufffd\ufffd", " ")
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(cap.lower())
    tokens_new = []
    for t in tokens:
        t = t.encode('ascii', 'ignore').decode('ascii')
        if len(t) > 0:
            tokens_new.append(t)
    caption.append(tokens_new)
    caption_new = []
    t = caption[0]
    rev = []
    for w in t:
        if w in train_dataset.wordtoix:
            rev.append(train_dataset.wordtoix[w])
    x, x_len = train_dataset.get_caption(rev)
    caption_new.append(np.squeeze(x, axis=1))
    return caption_new

def ID2sent(sample_caption):
    sentence = []
    for ID in sample_caption:
        if ID != train_dataset.ixtoword['<PAD>']:
            sentence.append(train_dataset.ixtoword[ID])
    return sentence

def get_random_int(min=0, max=10, number=5):
    return [random.randint(min,max) for p in range(0,number)]

def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1], 3))
    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w, :] = image
    return img

def imsave(images, size, path):
    return scipy.misc.imsave(path, merge(images, size))

def save_images(images, size, image_path):
    return imsave(images, size, image_path)

def threading_data(data=None, fn=None, **kwargs):
    def apply_fn(results, i, data, kwargs):
        results[i] = fn(data, **kwargs)
    results = [None] * len(data)
    threads = []
    for i in range(len(data)):
        t = threading.Thread(
                        name='threading_and_return',
                        target=apply_fn,
                        args=(results, i, data[i], kwargs)
                        )
        t.start()
        threads.append(t)
    for t in threads:
        t.join()
    return np.asarray(results)

def apply_transform(x, transform_matrix, channel_index=2, fill_mode='nearest', cval=0., order=1):
    x = np.rollaxis(x, channel_index, 0)
    final_affine_matrix = transform_matrix[:2, :2]
    final_offset = transform_matrix[:2, 2]
    channel_images = [ndi.interpolation.affine_transform(x_channel, final_affine_matrix,
                      final_offset, order=order, mode=fill_mode, cval=cval) for x_channel in x]
    x = np.stack(channel_images, axis=0)
    x = np.rollaxis(x, 0, channel_index + 1)
    return x

def transform_matrix_offset_center(matrix, x, y):
    o_x = float(x) / 2 + 0.5
    o_y = float(y) / 2 + 0.5
    offset_matrix = np.array([[1, 0, o_x], [0, 1, o_y], [0, 0, 1]])
    reset_matrix = np.array([[1, 0, -o_x], [0, 1, -o_y], [0, 0, 1]])
    transform_matrix = np.dot(np.dot(offset_matrix, matrix), reset_matrix)
    return transform_matrix

def rotation(x, rg=20, is_random=False, row_index=0, col_index=1, channel_index=2,
                    fill_mode='nearest', cval=0.):
    if is_random:
        theta = np.pi / 180 * np.random.uniform(-rg, rg)
    else:
        theta = np.pi / 180 * rg
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta), 0],
                                [np.sin(theta), np.cos(theta), 0],
                                [0, 0, 1]])
    h, w = x.shape[row_index], x.shape[col_index]
    transform_matrix = transform_matrix_offset_center(rotation_matrix, h, w)
    x = apply_transform(x, transform_matrix, channel_index, fill_mode, cval)
    return x

def crop(x, wrg, hrg, is_random=False, row_index=0, col_index=1, channel_index=2):
    h, w = x.shape[row_index], x.shape[col_index]
    assert (h > hrg) and (w > wrg), "The size of cropping should smaller than the original image"
    if is_random:
        h_offset = int(np.random.uniform(0, h-hrg) - 1)
        w_offset = int(np.random.uniform(0, w-wrg) - 1)
        return x[h_offset: hrg + h_offset ,w_offset: wrg + w_offset]
    else:
        h_offset = int(np.floor((h - hrg)/ 2.))
        w_offset = int(np.floor((w - wrg)/ 2.))
        h_end = h_offset + hrg
        w_end = w_offset + wrg
        return x[h_offset: h_end, w_offset: w_end]

def flip_axis(x, axis, is_random=False):
    if is_random:
        factor = np.random.uniform(-1, 1)
        if factor > 0:
            x = np.asarray(x).swapaxes(axis, 0)
            x = x[::-1, ...]
            x = x.swapaxes(0, axis)
            return x
        else:
            return x
    else:
        x = np.asarray(x).swapaxes(axis, 0)
        x = x[::-1, ...]
        x = x.swapaxes(0, axis)
        return x

def imresize(x, size=[100, 100], interp='bilinear', mode=None):
    if x.shape[-1] == 1:
        x = scipy.misc.imresize(x[:, :, 0], size, interp=interp, mode=mode)
        return x[:, :, np.newaxis]
    elif x.shape[-1] == 3:
        return scipy.misc.imresize(x, size, interp=interp, mode=mode)
    else:
        raise Exception("Unsupported channel %d" % x.shape[-1])

def prepro_img(x, mode=None):
    if mode=='train':
        x = flip_axis(x, axis=1, is_random=True)
        x = rotation(x, rg=16, is_random=True, fill_mode='nearest')
        x = imresize(x, size=[256 + 60, 256 + 60], interp='bilinear', mode=None)
        x = crop(x, wrg=256, hrg=256, is_random=True)
        x = x / (255. / 2.)
        x = x - 1.
    return x

def noisy(image):
    row,col,ch= image.shape
    mean = 0
    var = 0.001
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    gauss = gauss.reshape(row,col,ch)
    noisy = image + gauss
    return noisy

In [67]:
import tensorflow.contrib.slim as slim

def resBlock(x, channels=64, kernel_size=[3,3], scale=1):
    tmp = slim.conv2d(x, channels, kernel_size, activation_fn=None)
    tmp = tf.nn.relu(tmp)
    tmp = slim.conv2d(tmp, channels, kernel_size, activation_fn=None)
    tmp *= scale
    return x + tmp

def upsample(x, scale=2, features=64, activation=tf.nn.relu):
    assert scale in [2,3,4]
    x = slim.conv2d(x, features, [3,3], activation_fn=activation)
    if scale == 2:
        ps_features = 3*(scale**2)
        x = slim.conv2d(x, ps_features, [3,3], activation_fn=activation)
        x = PS(x, 2, color=True)
    elif scale == 3:
        ps_features =3*(scale**2)
        x = slim.conv2d(x, ps_features, [3,3], activation_fn=activation)
        x = PS(x, 3, color=True)
    elif scale == 4:
        ps_features = 3*(2**2)
        for i in range(2):
            x = slim.conv2d(x, ps_features, [3,3], activation_fn=activation)
            x = PS(x, 2, color=True)
    return x

def _phase_shift(I, r):
    bsize, a, b, c = I.get_shape().as_list()
    bsize = tf.shape(I)[0]
    X = tf.reshape(I, (bsize, a, b, r, r))
    X = tf.transpose(X, (0, 1, 2, 4, 3))
    X = tf.split(X, a, 1)
    X = tf.concat([tf.squeeze(x, axis=1) for x in X],2)
    X = tf.split(X, b, 1)
    X = tf.concat([tf.squeeze(x, axis=1) for x in X],2)
    return tf.reshape(X, (bsize, a*r, b*r, 1))

def PS(X, r, color=False):
    if color:
        Xc = tf.split(X, 3, 3)
        X = tf.concat([_phase_shift(x, r) for x in Xc],3)
    else:
        X = _phase_shift(X, r)
    return X

def log10(x):
    numerator = tf.log(x)
    denominator = tf.log(tf.constant(10, dtype=numerator.dtype))
    return numerator / denominator

def save(saver, sess, logdir, step):
    model_name = 'model.ckpt'
    checkpoint_path = os.path.join(logdir, model_name)
    if not os.path.exists(logdir):
        os.makedirs(logdir)
    saver.save(sess, checkpoint_path, global_step=step)
    print('The checkpoint has been created.')

def load(saver, sess, ckpt_path):
    saver.restore(sess, ckpt_path)
    print("Restored model parameters from {}".format(ckpt_path))

In [68]:
class EDSR(object):
    def __init__(self, img_size=64, num_layers=32, feature_size=256, scale=4, output_channels=3, lr=1e-4):
        print("Building EDSR...")
        """ Information """
        self.img_size = img_size
        self.scale = scale
        self.output_channels = output_channels
        self.lr = tf.Variable(lr, trainable=False)
        
        """ Place Holders"""
        self.input = x = tf.placeholder(tf.float32, [None, img_size, img_size, output_channels])
        self.target = y = tf.placeholder(tf.float32,[None, img_size*scale, img_size*scale, output_channels])
        image_input = x
        image_target = y
        
        """ Graph """
        x = slim.conv2d(image_input, feature_size, [3,3])
        conv_1 = x
        scaling_factor = 0.1
        for i in range(num_layers):
            x = resBlock(x, feature_size, scale=scaling_factor)
        x = slim.conv2d(x, feature_size, [3,3])
        x += conv_1
        x = upsample(x, scale, feature_size, None)
        self.out = output = tf.nn.tanh(x)
        
        """ Loss and Optimizer """
        self.loss = loss = tf.reduce_mean(tf.losses.absolute_difference(image_target, output))
        optimizer = tf.train.AdamOptimizer(self.lr)
        self.train_op = optimizer.minimize(self.loss)
        print("Building Complete!")

In [69]:
""" Training Parameters """
n_epochs = 200
batch_size = 16
lr = 2e-4
decay_every = 100
lr_decay = 0.5
train_samples_dir = 'EDSR_samples_with_noise_last'
if os.path.exists(train_samples_dir) == False:
    os.makedirs(train_samples_dir)
checkpoint_dir = './EDSR_checkpoint_last'

In [70]:
tf.reset_default_graph()
model = EDSR()

Building EDSR...
Building Complete!


In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
sess = tf.Session(config=config)
init = tf.global_variables_initializer()
sess.run(init)
saver = tf.train.Saver(var_list=tf.global_variables(), max_to_keep=10)
ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
if ckpt and ckpt.model_checkpoint_path:
    loader = tf.train.Saver(var_list=tf.global_variables())
    load_step = int(os.path.basename(ckpt.model_checkpoint_path).split('-')[1])
    load(loader, sess, ckpt.model_checkpoint_path)
else:
    print('no checkpoints find.')

print('[*] Training Start! [*]')
for epoch in range(n_epochs):
    start_time = time.time()
    if epoch !=0 and (epoch % decay_every == 0):
        new_lr_decay = lr_decay ** (epoch // decay_every)
        sess.run(tf.assign(model.lr, lr * new_lr_decay))
        log = " ** new learning rate: %f" % (lr * new_lr_decay)
        print(log)
    elif epoch == 0:
        log = " ** init lr: %f, decay_every_epoch: %d, lr_decay: %f" % (lr, decay_every, lr_decay)
        print(log)     
    n_batches = len(train_images) // batch_size
    random_list = np.random.permutation(len(train_images))
    for iteration in range(n_batches):
        step_time = time.time()
        idx = random_list[iteration*batch_size:(iteration+1)*batch_size]
        y = train_images[idx]
        y = threading_data(y, prepro_img, mode='train')
        x = []
        for image in y:
            eps = np.random.uniform()
            if eps < 0.5:
                x.append(noisy(resize(image, (64, 64, 3))))
            else:
                x.append(resize(image, (64, 64, 3)))
        x = np.asarray(x)
        assert x.shape[0] == y.shape[0]
        _, loss = sess.run([model.train_op, model.loss], feed_dict={model.input: x, model.target: y})
        if (iteration + 1) % (n_batches // 10) == 0:
            print('Iteration: [%3d/%3d] time: %4.4fs, loss: %.8f' % (iteration, n_batches, time.time() - step_time, loss))
    if (epoch + 1) % 1 == 0:
        print(" ** Epoch %d took %fs" % (epoch, time.time()-start_time))
        random_list = np.random.permutation(len(train_images))
        idx = random_list[0: batch_size]
        sample_images_64 = train_images_64[idx]
        save_images(sample_images_64, [4, 4], 'EDSR_samples_with_noise_last/train_64_{:02d}.png'.format(epoch))
        sample_outputs = sess.run([model.out], feed_dict={model.input: sample_images_64})
        sample_outputs = np.reshape(sample_outputs, [batch_size, 256, 256, 3])
        save_images(sample_outputs, [4, 4], 'EDSR_samples_with_noise_last/train_256_{:02d}.png'.format(epoch))  
    if (epoch != 0) and (epoch % 10) == 0:
        save(saver, sess, checkpoint_dir, epoch)
        print("[*] Save checkpoints SUCCESS!")
print('[*] Training Complete! [*]')

no checkpoints find.
[*] Training Start! [*]
 ** init lr: 0.000200, decay_every_epoch: 100, lr_decay: 0.500000


/home/chszerg/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/ipykernel_launcher.py:143: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


Iteration: [ 54/553] time: 1.3817s, loss: 0.31883496
Iteration: [109/553] time: 1.3506s, loss: 0.26690769
Iteration: [164/553] time: 1.4216s, loss: 0.20084368
Iteration: [219/553] time: 1.2972s, loss: 0.16904974
Iteration: [274/553] time: 1.4082s, loss: 0.14492038
Iteration: [329/553] time: 1.3450s, loss: 0.14118855
Iteration: [384/553] time: 1.4002s, loss: 0.12084224
Iteration: [439/553] time: 1.4322s, loss: 0.09217884
Iteration: [494/553] time: 1.3107s, loss: 0.08831292
Iteration: [549/553] time: 1.3936s, loss: 0.07825234
 ** Epoch 0 took 764.689144s


/home/chszerg/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/ipykernel_launcher.py:54: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Iteration: [ 54/553] time: 1.2753s, loss: 0.09919270
Iteration: [109/553] time: 1.4017s, loss: 0.09442442
Iteration: [164/553] time: 1.3933s, loss: 0.07722637
Iteration: [219/553] time: 1.3600s, loss: 0.09407873
Iteration: [274/553] time: 1.4329s, loss: 0.07246279
Iteration: [329/553] time: 1.3649s, loss: 0.07634988
Iteration: [384/553] time: 1.3731s, loss: 0.08511131
Iteration: [439/553] time: 1.3456s, loss: 0.08234745
Iteration: [494/553] time: 1.3697s, loss: 0.08057182
Iteration: [549/553] time: 1.4247s, loss: 0.06920260
 ** Epoch 1 took 766.512690s
Iteration: [ 54/553] time: 1.3945s, loss: 0.07378510
Iteration: [109/553] time: 1.3570s, loss: 0.07909963
Iteration: [164/553] time: 1.3802s, loss: 0.07130107
Iteration: [219/553] time: 1.3605s, loss: 0.08606105
Iteration: [274/553] time: 1.3742s, loss: 0.07748603
Iteration: [329/553] time: 1.4154s, loss: 0.06457392
Iteration: [384/553] time: 1.4001s, loss: 0.06934754
Iteration: [439/553] time: 1.3537s, loss: 0.06952020
Iteration: [494/5

...

Iteration: [329/553] time: 1.3997s, loss: 0.03937198
Iteration: [384/553] time: 1.3663s, loss: 0.04142757
Iteration: [439/553] time: 1.4220s, loss: 0.03366734
Iteration: [494/553] time: 1.3362s, loss: 0.03614082
Iteration: [549/553] time: 1.3729s, loss: 0.03220439
 ** Epoch 102 took 759.764726s
Iteration: [ 54/553] time: 1.3627s, loss: 0.03573419
Iteration: [109/553] time: 1.3221s, loss: 0.03968273
Iteration: [164/553] time: 1.3718s, loss: 0.03227775
Iteration: [219/553] time: 1.3432s, loss: 0.03603202
Iteration: [274/553] time: 1.3798s, loss: 0.03840365
Iteration: [329/553] time: 1.3857s, loss: 0.03450229
Iteration: [384/553] time: 1.4910s, loss: 0.03333214
Iteration: [439/553] time: 1.3806s, loss: 0.04413459
Iteration: [494/553] time: 1.3721s, loss: 0.03065227
Iteration: [549/553] time: 1.3446s, loss: 0.02735076
 ** Epoch 103 took 758.806869s
Iteration: [ 54/553] time: 1.3556s, loss: 0.03236112
Iteration: [109/553] time: 1.3996s, loss: 0.02940501
Iteration: [164/553] time: 1.4248s, l

Iteration: [ 54/553] time: 1.4083s, loss: 0.03324401
Iteration: [109/553] time: 1.3898s, loss: 0.03894841
Iteration: [164/553] time: 1.3485s, loss: 0.03430966
Iteration: [219/553] time: 1.3737s, loss: 0.03241329
Iteration: [274/553] time: 1.3819s, loss: 0.03952210
Iteration: [329/553] time: 1.3406s, loss: 0.03259883
Iteration: [384/553] time: 1.3791s, loss: 0.02891870
Iteration: [439/553] time: 1.3822s, loss: 0.03419856
Iteration: [494/553] time: 1.3952s, loss: 0.02939627
Iteration: [549/553] time: 1.4102s, loss: 0.02944275
 ** Epoch 117 took 760.081700s
Iteration: [ 54/553] time: 1.3560s, loss: 0.02992523
Iteration: [109/553] time: 1.4144s, loss: 0.03397071
Iteration: [164/553] time: 1.4331s, loss: 0.03803557
Iteration: [219/553] time: 1.3628s, loss: 0.02999526
Iteration: [274/553] time: 1.3701s, loss: 0.03297502
Iteration: [329/553] time: 1.3798s, loss: 0.03266442
Iteration: [384/553] time: 1.3980s, loss: 0.04106579
Iteration: [439/553] time: 1.3438s, loss: 0.03264390
Iteration: [494

Iteration: [329/553] time: 1.3398s, loss: 0.03249877
Iteration: [384/553] time: 1.3921s, loss: 0.03917490
Iteration: [439/553] time: 1.3902s, loss: 0.02744406
Iteration: [494/553] time: 1.3389s, loss: 0.03128971
Iteration: [549/553] time: 1.4097s, loss: 0.03293137
 ** Epoch 131 took 761.165954s
Iteration: [ 54/553] time: 1.3807s, loss: 0.03930164
Iteration: [109/553] time: 1.3993s, loss: 0.03344167
Iteration: [164/553] time: 1.3958s, loss: 0.03207651
Iteration: [219/553] time: 1.3429s, loss: 0.04069147
Iteration: [274/553] time: 1.3278s, loss: 0.03128942
Iteration: [329/553] time: 1.3638s, loss: 0.03429642
Iteration: [384/553] time: 1.3412s, loss: 0.03205210
Iteration: [439/553] time: 1.3788s, loss: 0.03763071
Iteration: [494/553] time: 1.3866s, loss: 0.03871727
Iteration: [549/553] time: 1.3954s, loss: 0.03210134
 ** Epoch 132 took 762.184458s
Iteration: [ 54/553] time: 1.4655s, loss: 0.03661067
Iteration: [109/553] time: 1.3882s, loss: 0.03613398
Iteration: [164/553] time: 1.3776s, l

Iteration: [ 54/553] time: 1.3489s, loss: 0.02695872
Iteration: [109/553] time: 1.4004s, loss: 0.02786917
Iteration: [164/553] time: 1.4238s, loss: 0.03208264
Iteration: [219/553] time: 1.3481s, loss: 0.02601837
Iteration: [274/553] time: 1.3937s, loss: 0.02834088
Iteration: [329/553] time: 1.4054s, loss: 0.03170127
Iteration: [384/553] time: 1.3693s, loss: 0.03178828
Iteration: [439/553] time: 1.3259s, loss: 0.03126133
Iteration: [494/553] time: 1.3848s, loss: 0.03228813
Iteration: [549/553] time: 1.3403s, loss: 0.02779342
 ** Epoch 146 took 762.229254s
Iteration: [ 54/553] time: 1.3497s, loss: 0.02981897
Iteration: [109/553] time: 1.3775s, loss: 0.02993159
Iteration: [164/553] time: 1.3404s, loss: 0.03063753
Iteration: [219/553] time: 1.3991s, loss: 0.03154993
Iteration: [274/553] time: 1.3694s, loss: 0.03548497
Iteration: [329/553] time: 1.3918s, loss: 0.03950241
Iteration: [384/553] time: 1.4436s, loss: 0.02886017
Iteration: [439/553] time: 1.3658s, loss: 0.02276262
Iteration: [494

Iteration: [384/553] time: 1.3226s, loss: 0.03012524
Iteration: [439/553] time: 1.3903s, loss: 0.03046297
Iteration: [494/553] time: 1.3709s, loss: 0.03091925
Iteration: [549/553] time: 1.3830s, loss: 0.03792283
 ** Epoch 160 took 759.241169s
The checkpoint has been created.
[*] Save checkpoints SUCCESS!
Iteration: [ 54/553] time: 1.3860s, loss: 0.03419601
Iteration: [109/553] time: 1.3608s, loss: 0.03287609
Iteration: [164/553] time: 1.3581s, loss: 0.03000726
Iteration: [219/553] time: 1.3435s, loss: 0.03242429
Iteration: [274/553] time: 1.4455s, loss: 0.03467091
Iteration: [329/553] time: 1.3827s, loss: 0.03466915
Iteration: [384/553] time: 1.3598s, loss: 0.03064938
Iteration: [439/553] time: 1.3200s, loss: 0.02696715
Iteration: [494/553] time: 1.2766s, loss: 0.03058861
Iteration: [549/553] time: 1.3199s, loss: 0.04251219
 ** Epoch 161 took 759.933005s
Iteration: [ 54/553] time: 1.3908s, loss: 0.03018094
Iteration: [109/553] time: 1.3693s, loss: 0.03473097
Iteration: [164/553] time: 